In [69]:
import numpy as np
import pandas as pd

# Provided data as a DataFrame
products_data = pd.DataFrame([
    {"D": 1000, "common_ordering_cost": 100, "specific_ordering_cost": 10, "unit_cost": 50, "H": 0.2},
    {"D": 300, "common_ordering_cost": 100, "specific_ordering_cost": 20, "unit_cost": 60, "H": 0.2},
    {"D": 100, "common_ordering_cost": 100, "specific_ordering_cost": 25, "unit_cost": 30, "H": 0.2},
    {"D": 50, "common_ordering_cost": 100, "specific_ordering_cost": 25, "unit_cost": 30, "H": 0.2},
])

# Calculate total ordering costs for each product
products_data['product_specific_ordering_costs'] = products_data['common_ordering_cost'] + products_data['specific_ordering_cost']

# Calculate holding costs for each product
products_data['holding_costs'] = products_data['H'] * products_data['unit_cost']

# Independent Sourcing Strategy
products_data['EOQ_independent'] = np.sqrt((2 * products_data['D'] * products_data['product_specific_ordering_costs']) / products_data['holding_costs'])
products_data['total_cost_independent'] = (products_data['D'] / products_data['EOQ_independent']) * products_data['product_specific_ordering_costs'] + (products_data['EOQ_independent'] / 2) * products_data['holding_costs']

# Output the results
print("Independent Sourcing Total Cost(Products are sourced independently):", products_data['total_cost_independent'].sum())


Independent Sourcing Total Cost(Products are sourced independently): 3073.9153138822376


In [14]:
import math
import pandas as pd

# Provided data as a DataFrame
products_data = pd.DataFrame([
    {"D": 1000, "common_ordering_cost": 100, "specific_ordering_cost": 10, "unit_cost": 50, "H": 0.2},
    {"D": 300, "common_ordering_cost": 100, "specific_ordering_cost": 20, "unit_cost": 60, "H": 0.2},
    {"D": 100, "common_ordering_cost": 100, "specific_ordering_cost": 25, "unit_cost": 30, "H": 0.2},
    {"D": 50, "common_ordering_cost": 100, "specific_ordering_cost": 25, "unit_cost": 30, "H": 0.2},
])

# Calculate the nominator
nominator = products_data['D'] * products_data['H'] * products_data['unit_cost']

# Aggregate all specific ordering costs to one common ordering cost
total_specific_ordering_cost = products_data['specific_ordering_cost'].sum()+products_data['common_ordering_cost'][0]


# Calculate n_star using the total effective ordering cost
n_star = math.sqrt(nominator.sum() / (2 * total_specific_ordering_cost))

# Calculate Optimal Order Size and Annual Holding Cost for each product
products_data['Optimal Order Size'] = products_data['D'] / n_star
products_data['Annual Holding Cost'] = products_data['Optimal Order Size'] * products_data['H'] * products_data['unit_cost'] / 2

# Calculate total annual cost for aggregation
annual_order_cost_aggregation = n_star * total_specific_ordering_cost
total_annual_cost_aggregation = annual_order_cost_aggregation + products_data['Annual Holding Cost'].sum()

# Set order frequency for each product
products_data['Order Frequency'] = n_star

# Display the modified DataFrame with the calculated values
print(products_data)

# Print the total annual cost for aggregation
print("Total Annual Cost (All four products are sourced with the same frequency):", total_annual_cost_aggregation)


      D  common_ordering_cost  specific_ordering_cost  unit_cost    H  \
0  1000                   100                      10         50  0.2   
1   300                   100                      20         60  0.2   
2   100                   100                      25         30  0.2   
3    50                   100                      25         30  0.2   

   Optimal Order Size  Annual Holding Cost  Order Frequency  
0          157.567719           787.838597         6.346478  
1           47.270316           283.621895         6.346478  
2           15.756772            47.270316         6.346478  
3            7.878386            23.635158         6.346478  
Total Annual Cost (All four products are sourced with the same frequency): 2284.7319317591728


In [89]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Given Data
data = {
    "D": [1000, 300, 100, 50],
    "common_ordering_cost": [100, 100, 100, 100],
    "specific_ordering_cost": [10, 20, 25, 25],
    "unit_cost": [50, 60, 30, 30],
    "H": [0.2, 0.2, 0.2, 0.2]
}

products_data = pd.DataFrame(data)

# Step 1: Independent Sourcing
specific_ordering_cost_total = products_data['specific_ordering_cost'].sum() + products_data['common_ordering_cost'].iloc[0]
products_data['EOQ_Independent'] = np.sqrt((2 * products_data['D'] * specific_ordering_cost_total) / (products_data['H'] * products_data['unit_cost']))
products_data['Order_Frequency_Independent'] = products_data['D'] / products_data['EOQ_Independent']


# Step 2: Tailored Aggregation - Identify specific ordering costs for other products
most_frequent_product_independent = products_data.loc[products_data['Order_Frequency_Independent'].idxmax()]
products_data['common_ordering_cost'] = 0
products_data['EOQ_Tailored_Step2'] = np.sqrt((2 * products_data['D'] * products_data['specific_ordering_cost']) / (products_data['H'] * products_data['unit_cost']))

# Set Order_Frequency_Tailored_Step2 for index 0 to be the same as Order_Frequency_Independent
products_data.at[0, 'Order_Frequency_Tailored_Step2'] = products_data.at[0, 'Order_Frequency_Independent']

# Calculate Order_Frequency_Tailored_Step2 for indices 1, 2, and 3
products_data['Order_Frequency_Tailored_Step2'][1:] = products_data['D'][1:] / products_data['EOQ_Tailored_Step2'][1:]

# Step 3: Identify frequency multiples (m_i)
products_data['m_i'] = np.ceil(most_frequent_product_independent['Order_Frequency_Independent'] / products_data['Order_Frequency_Tailored_Step2'])

# Step 4: Recalculate ordering frequency for all products
products_data['Order_Frequency_Step4'] = products_data['D'] / np.sqrt((2 * products_data['D'] * products_data['specific_ordering_cost']) / (products_data['H'] * products_data['unit_cost']))

# Step 5: Identify ordering frequency of all products
products_data['n'] = np.sqrt((2 * (products_data['H'] * products_data['unit_cost'] * products_data['D'] * products_data['m_i']).sum()) / (2 * (most_frequent_product_independent['common_ordering_cost'] + (products_data['specific_ordering_cost'] / products_data['m_i']).sum())))
products_data['Order_Frequency_Tailored_Step5'] = products_data['D'] / products_data['n']

# Calculate Total Annual Cost for Tailored Aggregation Step 5
products_data['Annual_Cost_Tailored_Step5'] = (products_data['D'] / products_data['Order_Frequency_Tailored_Step5']) * (most_frequent_product_independent['common_ordering_cost'] + products_data['specific_ordering_cost'] / products_data['m_i']) + 0.5 * products_data['EOQ_Tailored_Step2'] * products_data['H'] * products_data['unit_cost']

# Calculate Total Annual Cost for all products
total_annual_cost_tailored_step5 = products_data['Annual_Cost_Tailored_Step5'].sum()

# Display the product data in a table-like format
table_data = products_data[['D', 'common_ordering_cost', 'specific_ordering_cost', 'unit_cost', 'H', 'EOQ_Independent', 'Order_Frequency_Independent', 'EOQ_Tailored_Step2', 'Order_Frequency_Tailored_Step2', 'm_i', 'Order_Frequency_Step4', 'Order_Frequency_Tailored_Step5', 'Annual_Cost_Tailored_Step5']]
table = tabulate(table_data, headers='keys', tablefmt='grid', showindex=False)
print(table)

# Display the total annual cost for all products
print(f'Total Annual Cost (Tailored Aggregation Step 5): {total_annual_cost_tailored_step5}')


+------+------------------------+--------------------------+-------------+-----+-------------------+-------------------------------+----------------------+----------------------------------+-------+-------------------------+----------------------------------+------------------------------+
|    D |   common_ordering_cost |   specific_ordering_cost |   unit_cost |   H |   EOQ_Independent |   Order_Frequency_Independent |   EOQ_Tailored_Step2 |   Order_Frequency_Tailored_Step2 |   m_i |   Order_Frequency_Step4 |   Order_Frequency_Tailored_Step5 |   Annual_Cost_Tailored_Step5 |
+======+========================+==========================+=============+=====+===================+===============================+======================+==================================+=======+=========================+==================================+==============================+
| 1000 |                      0 |                       10 |          50 | 0.2 |          189.737  |                      5.270

<ipython-input-89-630a2c43de47>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_data['Order_Frequency_Tailored_Step2'][1:] = products_data['D'][1:] / products_data['EOQ_Tailored_Step2'][1:]
